In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import signal
from sklearn.metrics import mean_squared_error

### import data

In [ ]:
# appointments = pd.read_csv('Appointments.csv')
appointments = pd.read_csv('AppointmentsSince2015.csv')

In [ ]:
calls = pd.read_csv('CallsRingCentral.csv')

In [ ]:
reason_for_visit = pd.read_csv('MeetingReasonForVisits.csv')

In [ ]:
meeting_status = pd.read_csv('MeetingStatus.csv')

In [ ]:
offices = pd.read_csv('Offices.csv')

In [ ]:
providers_schedules = pd.read_csv('ProvidersSchedulesLastest.csv')

### combine/merge dataframes

In [ ]:
doctors = ['Psychiatry', 'Child & Adolescent Psychiatry', ]
RN_PAs = ['Medical', 'Psych/Mental Health, Child & Adolescent', 'Psych/Mental Health', 'Physician Assistant']
therapists = ['Marriage & Family Therapist', 'Psychologist', 'Specialist/Technologist, Other', 'Clinical' ]

In [ ]:
appointments['Specialty'].loc[appointments['Specialty'].isin(doctors)]= 'doctor'
appointments['Specialty'].loc[appointments['Specialty'].isin(RN_PAs)] = 'RN/PA'
appointments['Specialty'].loc[appointments['Specialty'].isin(therapists)] = 'therapist'

In [ ]:
merged1 = pd.merge(left=appointments, right=reason_for_visit, how='left', left_on='MeetingReasonForVisitId',\
                  right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'MeetingReasonForVisitId': 'ReasonForVisitId', 'Name':'ReasonForVisitName', 'Description':'ReasonForVisitDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
# merge in office name from offices df
merged1 = pd.merge(left=merged1, right=offices, how='left', left_on='OfficeId', right_on='id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'OfficeName', 'id_x':'id'})

In [ ]:
merged1.drop('id_y', axis=1, inplace=True)

In [ ]:
merged1 = pd.merge(left=merged1, right=meeting_status, how='left', left_on='MeetingStatusId', right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'MeetingStatusName', 'Description':'MeetingStatusDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
# reorder columns within the df
ordered_columns = ['id', 'Patient', 'PatientAgeMeetingDate', 'PatientGender',
       'PatientState', 'PatientCity', 'PatientInsurance', 'Provider',
       'Specialty', 'AppointmentDate', 'AppointmentCreated', 'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName',
       'ReasonForVisitDescription','MeetingStatusId', 'MeetingStatusName',
       'MeetingStatusDescription', 'OfficeId',  'OfficeName', 'CreatedBy']

In [ ]:
merged1 = merged1[ordered_columns]

### Data Cleaning: 
#### filling NaN values

In [ ]:
# filling NaN values in Specialty
implied_therapy = ['Therapy', 'New Patient Therapy', ]
implied_doctor = ['Therapy Telepsychiatry','Follow up Telepsychiatry', 'New Patient Therapy Telepsychiatry',\
                  'New Patient MD Adult', 'New Patient MD Adult Telepsychiatry']
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_therapy)] = 'therapist'
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_doctor)] = 'doctor'

In [ ]:
# convert date columns to datetime 
merged1['AppointmentCreated'] = pd.to_datetime(merged1['AppointmentCreated'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')
merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')

In [ ]:
# calculate time between AppointmentCreated and AppointmentDate
merged1['DaysFromAppointmentCreatedToVisit'] = (merged1['AppointmentDate'] - merged1['AppointmentCreated']).dt.days

In [ ]:
# merged1['Specialty'].isnull()
# merged1.isnull().sum()

In [ ]:
merged1 = merged1.set_index('AppointmentDate')

In [ ]:
merged2 = merged1.copy()

In [ ]:
# drop rows with missing specialty
merged2.dropna(subset=['Specialty'], how='all', inplace=True)

In [ ]:
merged2 = merged2['2018-02-28':]

In [ ]:
test_data = merged1['2018-04-30':'2018-02-28']

### Use the number of hours per day per provider/provider specialty

In [ ]:
merged2['DurationHours'] = merged2['AppointmentDuration'] /60

In [ ]:
merged2['AppointmentDate'] = pd.to_datetime(merged2.index,format='%Y-%m-%d')

In [ ]:
duration_df = merged2[['Provider', 'Specialty', 'AppointmentCreated', 'AppointmentDate', 'AppointmentDuration',
       'ReasonForVisitName', 'DurationHours', 'ReasonForVisitDescription','MeetingStatusName', 'MeetingStatusDescription',
       'OfficeId']]

In [ ]:
# drop appointments that are longer than 90 minutes
duration_df = duration_df[duration_df['AppointmentDuration'] <= 90]

In [ ]:
# drop remaining columns with missing values
duration_df.dropna(axis=0, inplace=True)

In [ ]:
doctors = duration_df[duration_df['Specialty'] == 'doctor']
therapists = duration_df[duration_df['Specialty'] == 'therapist']
RN_PA = duration_df[duration_df['Specialty'] == 'RN/PA']

In [ ]:
doc_duration = doctors.groupby(doctors.index.date)['DurationHours'].sum()
RN_PA_duration = RN_PA.groupby(RN_PA.index.date)['DurationHours'].sum()
therapist_duration = therapists.groupby(therapists.index.date)['DurationHours'].sum()

In [ ]:
def plot_series_and_differences(series, ax, num_diff, title):
    "Plot raw data and specified number of differences"
    ax[0].plot(series.index, series)
    ax[0].set_title('Raw series: {}'.format(title))
    for i in range(1, num_diff+1):
        diff = series.diff(i)
        ax[i].plot(series.index, diff)
        ax[i].set_title('Difference # {}'.format(str(i)))   

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=doc_duration, ax=axes, num_diff=2, title='Doctors')
fig.tight_layout()

In [ ]:
def run_augmented_Dickey_Fuller_test(series, num_diffs=None):
    test = sm.tsa.stattools.adfuller(series)
    if test[1] >= 0.05:
        print('The p-value for the series is: {p}, which is not significant'.format(p=test[1]))
    else:
        print('The p-value for the series is: {p}, which is significant'.format(p=test[1]))  
    if num_diffs:
        for i in range(1, num_diffs +1):
            test = sm.tsa.stattools.adfuller(series.diff(i)[i:])
            if test[1] >= 0.05:
                print('The p-value for difference {diff} is: {p}, which is not significant'.format(diff=str(i), p=test[1]))
            else:
                print('The p-value for difference {diff} is: {p}, which is significant'.format(diff=str(i), p=test[1]))   

In [ ]:
# test for stationarity of doctors data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(doc_duration, num_diffs=2)

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=RN_PA_duration, ax=axes, num_diff=2, title='RN/PA')
fig.tight_layout()

In [ ]:
# test for stationarity of RN/PA data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(RN_PA_duration, num_diffs=2)

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=therapist_duration, ax=axes, num_diff=2, title='Therapists')
fig.tight_layout()

In [ ]:
# test for stationarity of therapist data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(therapist_duration, num_diffs=2)

In [ ]:
def plot_autocorrelation(series, params, lags, alpha, title):
    plt.rcParams.update(params)
    acf_plot = tsaplots.plot_acf(series, lags=lags, alpha=alpha)
    plt.title(title)
    plt.xlabel('Number of Lags')
    plt.show()

def plot_partial_autocorrelation(series, params, lags, alpha, title):
    plt.rcParams.update(params)
    acf_plot = tsaplots.plot_pacf(series, lags=lags, alpha=alpha)
    plt.xlabel('Number of Lags')
    plt.title(title)
    plt.show()

In [ ]:
d_ts_index = pd.to_datetime(doc_duration.index)
RN_ts_index = pd.to_datetime(RN_PA_duration.index)
t_ts_index = pd.to_datetime(therapist_duration.index)

In [ ]:
doc_duration.index = d_ts_index
RN_PA_duration.index = RN_ts_index
therapist_duration.index = t_ts_index

In [ ]:
def plot_decomposition(series, params, freq, title):
    "Plots observed, trend, seasonal, residual"
    plt.rcParams.update(params)
    decomp = sm.tsa.seasonal_decompose(series, freq=freq)
    fig = decomp.plot()
    plt.title(title)
    plt.show()

In [ ]:
params = {'figure.figsize': [8, 8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(doc_duration, params=params, freq=31, title='Doctors Decomposition')

In [ ]:
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(RN_PA_duration, params=params, freq=31, title='RN/PA Decomposition')

In [ ]:
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(therapist_duration, params=params, freq=31, title='Therapist Decomposition')

#### Weekly Doctors Hours

In [ ]:
# downsample from daily to weekly data, filling missing data w/ the mean
weekly_doc_dur = doc_duration.resample(rule='W').mean() # weekly time spent

In [ ]:
weekly_doc_dur.fillna(method='bfill', inplace=True)

In [ ]:
weekly_doc_dur.tail()

In [ ]:
plot_series(weekly_doc_dur, xlabel='Date', ylabel='Hours', plot_name='Doctor Hours per Week')

In [ ]:
# determine the order of the AR(p) model w/ partial autocorrelation function, alpha=width of CI
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_partial_autocorrelation(weekly_doc_dur, params=params, lags=30, alpha=0.05, \
    title='Weekly Doctor Hours Partial Autocorrelation')
## lag/order = 5 should work

In [ ]:
def get_AR_model(data, order):
    model = ARMA(data, order=order)
    results = model.fit()
    return results

In [ ]:
def plot_AR_model(data, order, start, end, title='', xlabel='', ylabel=''):
    results = get_AR_model(data, order)
    results.plot_predict(start=start, end=end)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()

In [ ]:
start=0
end=180
title = 'Doctors Hours Spent by Week (MA)'
xlabel = 'Number of Hours'
ylabel = 'Date'
plot_AR_model(data=weekly_doc_dur, order=(5,0), start=start, end=end, \
              title=title, xlabel=xlabel, ylabel=ylabel)

In [ ]:
# check goodness of fit for a range of parameters for AR model
def get_AR_model_order_BIC(data, max_order_plus_one):
    "Calculates Baysian Information Criterion for range of model orders"
    BIC_array = np.zeros(max_order_plus_one)
    for p in range(1, max_order_plus_one):
        results = get_AR_model(data, order=(p,0))
        BIC_array[p] = results.bic
    return BIC_array

In [ ]:
def plot_BIC_AR_model(data, max_order_plus_one):
    "Plots BIC for range of orders"
    array = get_AR_model_order_BIC(data, max_order_plus_one)
    plt.plot(range(1, max_order_plus_one), array[1:max_order_plus_one], marker='o')
    plt.xlabel('Order of {mod} Model'.format(mod='AR'))
    plt.ylabel('Baysian Information Criterion')
    plt.show()

In [ ]:
# plot information criteria for different orders
plot_BIC_AR_model(data=weekly_doc_dur, max_order_plus_one=10)

#### MA model of doctors weekly hours data

In [ ]:
def get_MA_model(data, order):
    model = ARMA(data, order=order)
    results = model.fit()
    return results

In [ ]:
def plot_MA_model(data, order, start, end, title='', xlabel='', ylabel=''):
    results = get_MA_model(data, order)
    results.plot_predict(start=start, end=end)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()

In [ ]:
start=0
end=180
title = 'Doctors Hours Spent by Week (MA)'
xlabel = 'Number of Hours'
ylabel = 'Date'
plot_MA_model(data=weekly_doc_dur, order=(0,5), start=start, end=end, \
              title=title, xlabel=xlabel, ylabel=ylabel)

In [ ]:
# check goodness of fit for a range of parameters for MA model
def get_MA_model_order_BIC(data, order, max_order_plus_one):
    "Calculates Baysian Information Criterion for range of model orders"
    BIC_array = np.zeros(max_order_plus_one)
    for p in range(1, max_order_plus_one):
        results = get_MA_model(data, order)
        BIC_array[p] = results.bic
    return BIC_array

In [ ]:
def plot_BIC_MA_model(data, order, max_order_plus_one):
    "Plots BIC for range of orders"
    array = get_MA_model_order_BIC(data, max_order_plus_one)
    plt.plot(range(1, max_order_plus_one), array[1:max_order_plus_one], marker='o')
    plt.xlabel('Order of {mod} Model'.format(mod='ARMA'))
    plt.ylabel('Baysian Information Criterion')
    plt.show()

In [ ]:
# autocorrelation function for MA model
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_autocorrelation(weekly_doc_dur, params=params, lags=30, alpha=0.05, \
    title='Weekly Doctor Hours Autocorrelation')

#### ARIMA model doctors weekly hours data

In [ ]:
def get_ARIMA_Model(data, order):
    "Fits ARIMA model"
    arima = ARIMA(data, order=order)
    results = arima.fit()
    summary = results.summary()
    params = results.params
    residuals = results.resid
    return results, summary, params, residuals

In [ ]:
def plot_ARIMA_model(data, order, start, end, title='', xlabel='', ylabel=''):
    "Plots ARIMA model"
    results, summary, params, residuals = get_ARIMA_Model(data, order)
    fig = results.plot_predict(start=start, end=end)
    plt.title(title)
    plt.ylabel(xlabel)
    plt.xlabel(ylabel)
    plt.show()

In [ ]:
order=(5,1,5)
data = weekly_doc_dur
plot_ARIMA_model(data=data, order=order, start=1, end=180)


In [ ]:
results, summary, params, residuals = get_ARIMA_Model(weekly_doc_dur, (5,1,5))
arima_residuals = residuals

In [ ]:
def plot_ARIMA_resids(data, order, start, end, title='', xlabel='', ylabel=''):
    "Plots ARIMA model residuals"
    results, summary, params, residuals = get_ARIMA_Model(data, order)
    residuals.plot(figsize=(5,5))
    plt.title(title)
    plt.ylabel(xlabel)
    plt.xlabel(ylabel)
    plt.show()

In [ ]:
order=(5,1,5)
data = weekly_doc_dur
plot_ARIMA_resids(data=data, order=order, start=1, end=180, title='ARIMA residuals')

In [ ]:
results, summary, params, residuals = get_ARIMA_Model(data=weekly_doc_dur,order=(5,1,5))
residuals.plot(kind='kde', figsize=(5,5))
plt.title('ARIMA residuals KDE')
plt.xlabel('Residual Error')
plt.show()

In [ ]:
arima_residuals.describe()

#### prepare test data for hours spent

In [ ]:
# for specialty = doctor
dr_test_data = test_data[test_data['Specialty'] == 'doctor']

In [ ]:
dr_test_duration = dr_test_data['AppointmentDuration']

In [ ]:
dr_test_duration_weekly = dr_test_duration.resample(rule='W').mean()

In [ ]:
dr_test_duration_weekly.fillna(method='bfill', inplace=True)

In [ ]:
def get_rolling_ARIMA_forecast(train_data, test_data, order):
    "Calculates rolling ARIMA forecast, returns predicted vs actual"
    history = [x for x in train]
    predictions = []
    for t in range(len(test)):
        arima = ARIMA(history, order=order)
        arima_fitted = arima.fit()
        forecast = arima_fitted.forecast()
        yhat = forecast[0]
        predictions.append(yhat)
        observed = test[t]
        history.append(observed)
    return predictions, test

In [ ]:
train = weekly_doc_dur.values
test = dr_test_duration_weekly.values
order = (5,1,0)
predicted, expected = get_rolling_ARIMA_forecast(train_data=train, test_data=test, order=order)

In [ ]:
def plot_rolling_ARIMA_forecast_w_legend(train_data, test_data, order, title):
    "Calculates and plots rolling ARIMA forecast"
    predicted, expected = get_rolling_ARIMA_forecast(train_data, test_data, order)
    predictions = np.hstack(predicted)
    df = pd.DataFrame({'predicted': predictions, 'actual':test})
    df.plot()
    plt.title(title)
    plt.show()

In [ ]:
mse = mean_squared_error(predicted, expected)
mse

In [ ]:
train = weekly_doc_dur.values
test = dr_test_duration_weekly.values
order = (5,1,0)
plot_rolling_ARIMA_forecast_w_legend(train_data=train, test_data=test, order=order,\
                    title='Dr. Hours Spent, Predicted vs. Actual')